1. Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
import requests
import json
from lxml import html
from bs4 import BeautifulSoup as bs
from time import sleep
import pandas as pd

In [2]:
#функция парсинга сайта hh.ru

def page_for_parsing(vacancy_name, num_page):

    url = 'https://hh.ru/search/vacancy'

    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
            }

    params = {'text': vacancy_name, 'page': num_page}

    return requests.get(url=url, headers=headers, params=params)

In [20]:
#функция определения страниц в поиске

def max_find_page(button_whith_number_find_pages):
    return max([int(el) for el in button_whith_number_find_pages if el.isdigit()])           

In [4]:
#функция определения минимальной и максимальной зарплаты

def min_max_salary(salary_data):
    
    currency_str=salary_data
    salary_data.replace(" ", "")
    min_salary, max_salary = '', ''
    
    if salary_data[0].isdigit()or salary_data[0]=='о':
        j = 1
    elif salary_data[0]=='д':
        j = 2
    
    for el in salary_data:
        if j==1 and el.isdigit():
            min_salary += el
        elif el=='–':
            j = 2
        if j==2 and el.isdigit():
            max_salary += el
            
    if min_salary == '':
        min_salary = None
        
    if max_salary == '':
        max_salary = None
        
    currency = currency_str[currency_str.rfind(" ")+1:]
    
    return [min_salary, max_salary, currency]
            

In [5]:
#функция формирования данных по вакансии

def vacancy_data(information):
    vacancy_name = information.find('a', {'class':'serp-item__title'}).text
    vacancy_url = information.find('a', {'class':'serp-item__title'}, href=True)['href']
    try:
        salary_data = information.find('span', {'class':'bloko-header-section-3'}).text
    except:
        salary_data = None
    try:
        employer = information.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}).text
    except:
        employer = None
    try:
        employer_url = 'https://hh.ru'+information.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}, href=True)['href']
    except:
        employer_url = None
    try:
        adress = information.find('div', {'data-qa':'vacancy-serp__vacancy-address'}).text
    except:
        employer_url = None
    try:
        short_description_of_vacancy = (information.find('div', {'data-qa':'vacancy-serp__vacancy_snippet_responsibility'}).text +' '
                                    +information.find('div', {'data-qa':'vacancy-serp__vacancy_snippet_requirement'}).text)
    except:
        short_description_of_vacancy = None
        
    if salary_data==None:
        salary_list = [None, None, None]
    else:
        salary_list = min_max_salary(salary_data)
    
    vacancy_dict = {
        'vacancy_name':vacancy_name,
        'vacancy_url':vacancy_url, 
        'min_salary': salary_list[0],
        'max_salary': salary_list[1],
        'currency': salary_list[2],
        'employer': employer,
        'employer_url':employer_url,
        'adress': adress,
        'short_description_of_vacancy': short_description_of_vacancy}
    
    return vacancy_dict

In [6]:
vacancy_name = input('Введите название вакансии для поиска: ')

Введите название вакансии для поиска: product manager


In [7]:
first_search_page = page_for_parsing(vacancy_name, "0")

first_soup = bs(first_search_page.content, 'html.parser')
first_dom = html.fromstring(first_search_page.text)

In [8]:
button_whith_number_find_pages = first_dom.xpath("//a[@class='bloko-button']/span/text()")

max_page = max_find_page(button_whith_number_find_pages)

In [9]:
vacancy_infromation_soup = first_soup.find_all('div', {'class':'serp-item'})
vacances_list = []

for el in vacancy_infromation_soup:
    vacances_list.append(vacancy_data(el))
    

for num_page in range(1, max_page):
    sleep(2)
    search_page = page_for_parsing(vacancy_name, num_page)
    soup = bs(search_page.content, 'html.parser')
    vacancy_infromation_soup = soup.find_all('div', {'class':'serp-item'})
    for el in vacancy_infromation_soup:
        vacances_list.append(vacancy_data(el))


In [11]:
name = 'data ' + vacancy_name + '.json'

with open(name, 'w', encoding='utf-8') as outfile:
    json.dump(vacances_list, outfile, ensure_ascii=False)

In [12]:
df = pd.DataFrame(vacances_list)

df

,vacancy_name,vacancy_url,min_salary,max_salary,currency,employer,employer_url,adress,short_description_of_vacancy
0,Product manager в направление работодатели,https://hh.ru/vacancy/76495464?from=vacancy_se...,None,None,None,ООО HeadHunter::Product Management,https://hh.ru/employer/1455?dpt=hh-1455-produc...,"Москва, Алексеевская","Формулировать продуктовые гипотезы, которые не..."
1,Senior Product manager в Talantix,https://hh.ru/vacancy/72542824?from=vacancy_se...,None,None,None,ООО HeadHunter::Product Management,https://hh.ru/employer/1455?dpt=hh-1455-produc...,"Москва, Алексеевская",Разработать стратегическое видение и план разв...
2,Менеджер продукта (Product manager),https://hh.ru/vacancy/77073615?from=vacancy_se...,None,None,None,ЦРТ | Группа компаний,https://hh.ru/employer/4585?hhtmFrom=vacancy_s...,"Москва, Курская и еще 3",Анализ огромного количества текстовых данных (...
3,Senior Product manager / Руководитель направле...,https://hh.ru/vacancy/75956286?from=vacancy_se...,None,None,None,ООО HeadHunter::Product Management,https://hh.ru/employer/1455?dpt=hh-1455-produc...,"Москва, Алексеевская",Разработать стратегическое видение и планы раз...
4,Менеджер продукта (Product manager),https://hh.ru/vacancy/77073597?from=vacancy_se...,None,None,None,ЦРТ | Группа компаний,https://hh.ru/employer/4585?hhtmFrom=vacancy_s...,"Санкт-Петербург, Выборгская и еще 2",Анализ огромного количества текстовых данных (...
...,...,...,...,...,...,...,...,...,...
795,Product Designer (DAM),https://hh.ru/vacancy/76092617?from=vacancy_se...,None,None,None,Самокат (ООО Умное пространство),https://hh.ru/employer/5488399?hhtmFrom=vacanc...,Москва,Что нужно будет делать: Вместе с менеджером пр...
796,Менеджер продукта (ML),https://hh.ru/vacancy/76391987?from=vacancy_se...,None,None,None,"VK, Рекламные технологии",https://hh.ru/employer/15478?dpt=mailru-15478-...,"Москва, Аэропорт","В рекламный департамент VK, в направление инте..."
797,Менеджер по продукту СТМ,https://hh.ru/vacancy/75520237?from=vacancy_se...,150000,None,руб.,Кадровое Агентство Уникальных Специалистов,https://hh.ru/employer/98302?hhtmFrom=vacancy_...,"Москва, Калужская",Разработка и вывод новых продуктов на рынок; к...
798,Менеджер по развитию продаж,https://hh.ru/vacancy/76030017?from=vacancy_se...,100000,None,руб.,ООО АйПиМатика,https://hh.ru/employer/1681293?hhtmFrom=vacanc...,"Москва, Сходненская",Участие в развитии и осуществлении проектных п...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   vacancy_name                  800 non-null    object
 1   vacancy_url                   800 non-null    object
 2   min_salary                    234 non-null    object
 3   max_salary                    141 non-null    object
 4   currency                      271 non-null    object
 5   employer                      797 non-null    object
 6   employer_url                  797 non-null    object
 7   adress                        800 non-null    object
 8   short_description_of_vacancy  727 non-null    object
dtypes: object(9)
memory usage: 56.4+ KB
